In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
## for colab only
## !!!!! Make sure to set up GPU in colab notebook !!!!!! 
## https://colab.research.google.com/drive/1P7okDVh6viCIOkii6UAF2O9sTAcKGNWq
# import os
# if not os.path.exists('functional'):
#   !git clone https://github.com/nd7141/functional.git
# %cd functional

# !pip install dgl-cu101 catboost category_encoders

In [6]:
import pandas as pd
import networkx as nx
import random
import numpy as np
import importlib
import sys

from utils import get_masks
from collections import defaultdict as ddict

import torch
from sklearn.manifold import TSNE

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import dgl 
import ast 
import json 
import lightgbm

In [65]:
# vk data (last_seen_time target)
target = 'age'
data = 'vk'
X = pd.read_csv(f'data/{data}/{target}/X.csv')
y = pd.read_csv(f'data/{data}/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/{data}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
cat_features = np.where(X.columns.isin(['country', 'city', 'has_mobile', 'last_seen_platform', 
                                         'political', 'religion_id', 'alcohol', 'smoking', 
                                         'relation', 'sex', 'university']))[0]

X.iloc[:, cat_features] = X.iloc[:, cat_features].astype(str) # catboost accept only str or int as cat feature

with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)
    
seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=0)

print('Original data size')
X.shape, y.shape, networkx_graph.order(), networkx_graph.size()

Original data size


((54028, 14), (54028, 1), 54028, 213644)

In [9]:
seed_masks = dict()
for seed in range(5):
    train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=seed)
    seed_masks[seed] = {
        'train': train_mask,
        'val': val_mask,
        'test': test_mask
    }

with open(f'data/{data}/{target}/masks.json', 'w+') as fp:
    json.dump(seed_masks, fp)

In [20]:
# wikipedia
target = 'squirrel'
data = 'wikipedia'
X = pd.read_csv(f'data/{data}/{target}/X.csv')
y = pd.read_csv(f'data/{data}/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/{data}/{target}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
cat_features = []

with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)
    
seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=1)

print('Original data size')
X.shape, y.shape, networkx_graph.order(), networkx_graph.size()

print(f'{name}')
print(f'{networkx_graph.order()}')
print(f'{networkx_graph.size()}')
print(f'{X.shape[1]}')

Original data size
squirrel
5201
198493
3148


In [33]:
# twitch
target='PTBR' # ['DE', 'ENGB', 'ES', 'FR', 'RU']
data = 'twitch'

X = pd.read_csv(f'data/twitch/{target}/X.csv')
y = pd.read_csv(f'data/twitch/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/twitch/{target}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
cat_features = []

with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)

seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=1)

print('Original data size')
print(target, X.shape, y.shape, networkx_graph.order(), networkx_graph.size())

Original data size
DE (9498, 2514) (9498, 1) 9498 153138
Original data size
ENGB (7126, 2545) (7126, 1) 7126 35324
Original data size
ES (4648, 2148) (4648, 1) 4648 59382
Original data size
FR (6549, 2275) (6551, 1) 6549 112666
Original data size
RU (4385, 2224) (4385, 1) 4385 37304


In [64]:
# county
target='unemployment'
# target='bachelor'
data = 'county'

X = pd.read_csv(f'data/{data}/{target}/X.csv')
y = pd.read_csv(f'data/{data}/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/{data}/{target}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
cat_features = []
    
with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)

seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=2)

print('Original data size')
X.shape, y.shape, networkx_graph.order(), networkx_graph.size()

print(f'{target}')
print(f'{networkx_graph.order()}')
print(f'{networkx_graph.size()}')
print(f'{X.shape[1]}')

Original data size
unemployment
3217
12684
7


In [63]:
# power
data = 'power'
target='texas'

X = pd.read_csv(f'data/{data}/{target}/X.csv')
y = pd.read_csv(f'data/{data}/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/{data}/{target}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
cat_features = []
    
with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)

seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=2)

print('Original data size')
X.shape, y.shape, networkx_graph.order(), networkx_graph.size()



Original data size


((2000, 5), (2000, 1), 2000, 1918)

In [62]:
# house data
data = 'house'
target = ''
X = pd.read_csv(f'data/{data}/{target}/X.csv')
y = pd.read_csv(f'data/{data}/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/{data}/{target}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
for (n1,n2) in networkx_graph.edges(data=False):# remove edge attributes
    networkx_graph[n1][n2].pop('weight', None)  
cat_features = None
    
with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)

seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=42)

print('Original data size')
print(X.shape, y.shape, networkx_graph.order(), networkx_graph.size())



Original data size
(20640, 6) (20640, 1) 20640 182146


In [37]:
# ctr
data = 'ctr'
target = ''
X = pd.read_csv(f'data/{data}/{target}/X.csv')
y = pd.read_csv(f'data/{data}/{target}/y.csv')
networkx_graph = nx.read_graphml(f'data/{data}/{target}/graph.graphml')
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))})
cat_features = np.where(X.columns.isin(['C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']))[0] 
    
with open(f'data/{data}/{target}/masks.json') as f:
    masks = json.load(f)

seed = '0'
train_mask, val_mask, test_mask = masks[seed]['train'], masks[seed]['val'], masks[seed]['test']

# train_mask, val_mask, train_val_mask, test_mask = get_masks(X.shape[0], train_size = 0.6, val_size = 0.2, random_seed=1)

print('Original data size')
X.shape, y.shape, networkx_graph.order(), networkx_graph.size()

Original data size


((1297, 9), (1297, 1), 1297, 54364)

In [168]:
import models.GBDT 
import sys
import importlib
importlib.reload(sys.modules['models.GBDT'])
from models.GBDT import GBDTLGBM

model = GBDTLGBM(task='regression', learning_rate=0.1, num_leaves=31, max_bin=255, lambda_l1=0., lambda_l2=0., boosting='gbdt')

min_rmse_epoch, metrics = model.fit(X, y, train_mask, val_mask, test_mask,
            cat_features=cat_features, num_epochs=1000, patience=100,
            learning_rate=None, plot=True, verbose=False,
            loss_fn="lgbm.txt", metric_name='loss')

mlp.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='LGBM RMSE', metric_name='loss')

Training until validation scores don't improve for 100 rounds
[100]	training's l2: 40.0772	training's rmse: 6.33065	valid_1's l2: 56.1042	valid_1's rmse: 7.49027	valid_2's l2: 51.6419	valid_2's rmse: 7.18623
Early stopping, best iteration is:
[25]	training's l2: 46.474	training's rmse: 6.81718	valid_1's l2: 55.8095	valid_1's rmse: 7.47057	valid_2's l2: 51.2415	valid_2's rmse: 7.15832
Finished training. Total time: 0.56
Best loss at iteration 24: 6.817/7.471/7.158


In [170]:
import models.GBDT 
import sys
import importlib
importlib.reload(sys.modules['models.GBDT'])
from models.GBDT import GBDTCatBoost

gbdt = GBDTCatBoost(task='regression', depth=6, learning_rate=0.1, l2_leaf_reg=0., max_bin=254)

min_rmse_epoch, accuracies = gbdt.fit(X, y, train_mask, val_mask, test_mask,
            cat_features=cat_features, num_epochs=1000, patience=100,
            plot=True, verbose=False,
            loss_fn="gbdt.txt", metric_name='loss')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Finished training. Total time: 12.13 | Number of trees: 237 | Time per tree: 0.05
Best loss at iteration 329: 6.721/7.417/7.100


In [171]:
import models.MLP 
import models.Base
import sys
import importlib 
importlib.reload(sys.modules['models.Base'])
importlib.reload(sys.modules['models.MLP']) # these imports are necessary only for local changes to the file
from models.MLP import MLP

num_epochs = 1000
patience = 500
learning_rate = 0.01

mlp = MLP(task='regression')
min_rmse_epoch, metrics = mlp.fit(X, y, train_mask, val_mask, test_mask, cat_features=cat_features, num_epochs=num_epochs,
            learning_rate=learning_rate, hidden_dim=128, patience=500, logging_epochs=1, loss_fn="mlp.txt", metric_name='loss')
# mlp.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='R2 RMSE', metric_name='accuracy', descending=False)
mlp.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='MLP RMSE', metric_name='loss', start_from=100)

Epoch 00999 | Loss 8.342 | Loss 7.302/7.606/7.336 | Time 0.0109: 100%|██████████| 1000/1000 [00:11<00:00, 87.48it/s]  


Best loss at iteration 955: 7.274/7.578/7.308


In [175]:
# GNN
import models.GNN
import sys
importlib.reload(sys.modules['models.Base'])
importlib.reload(sys.modules['models.GNN'])
from models.GNN import GNN

# names = ['gat', 'gcn', 'cheb', 'agnn', 'appnp']
names = ['gat']

num_epochs = 5000
patience = 500
learning_rate = 0.01

use_mlp = False # whether to use MLP in front of the model (end-to-end)
append_mlp = False # whether to append MLP predictions to original features (or keep only predictions)
optimize_node_features=False # whether to optimize final resulted node features (i.e. with or without MLP predictions)

for name in names:
    print(name)
    if name == 'gat':
        hidden_dim = 8
    else:
        hidden_dim = 64

    # Note that GAT has 8 heads, with each having 8 hidden dim, so the final embedding size is 8*8
    gnn = GNN(heads=8, dropout=0, task='regression', name=name, lang='dgl', use_mlp=use_mlp, join_with_mlp=append_mlp)

    min_rmse_epoch, metrics = gnn.fit(networkx_graph, X, y, train_mask, val_mask, test_mask, num_epochs=num_epochs,
                cat_features=cat_features, optimize_node_features=optimize_node_features, learning_rate=learning_rate, patience=patience, logging_epochs=1, hidden_dim=hidden_dim,
#                 gbdt_predictions=gbdt.model.predict(X),
                loss_fn="gnn.txt", metric_name='loss', normalize_features=True, replace_na=True)

# gnn.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='R2 RMSE', metric_name='accuracy', descending=False)
    gnn.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='GNN RMSE', metric_name='loss', start_from=100)

gat


Epoch 04999 | Loss 7.004 | Loss 7.004/7.408/7.163 | Time 0.0217: 100%|██████████| 5000/5000 [02:13<00:00, 37.33it/s]  


Best loss at iteration 4999: 7.004/7.408/7.163


In [177]:
# Res-GNN
import models.GNN
import sys
importlib.reload(sys.modules['models.Base'])
importlib.reload(sys.modules['models.GNN'])
from models.GNN import GNN

num_epochs = 1000
patience = 500
learning_rate = 0.01

use_mlp = False # whether to use MLP in front of the model (end-to-end)
append_mlp = False # whether to append MLP predictions to original features (or keep only predictions)
optimize_node_features=False # whether to optimize final resulted node features (i.e. with or without MLP predictions)
only_gbdt = False # use only GBDT features as input features to the end-to-end model (or append GBDT features to original features)

# Note that GAT has 8 heads, with each having 8 hidden dim, so the final embedding size is 8*8
gnn = GNN(heads=8, dropout=0, task='regression', name='gat', lang='dgl', use_mlp=use_mlp, join_with_mlp=append_mlp)

min_rmse_epoch, metrics = gnn.fit(networkx_graph, X, y, train_mask, val_mask, test_mask, num_epochs=num_epochs,
            cat_features=cat_features, optimize_node_features=optimize_node_features, learning_rate=learning_rate, patience=patience, logging_epochs=1, hidden_dim=8,
            gbdt_predictions=gbdt.model.predict(X), only_gbdt=only_gbdt,
            loss_fn="gnn.txt", metric_name='loss', normalize_features=True, replace_na=True)

# gnn.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='R2 RMSE', metric_name='accuracy', descending=False)
gnn.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='Res-GNN RMSE', metric_name='loss', start_from=100)

Epoch 00999 | Loss 6.848 | Loss 6.848/7.312/7.038 | Time 0.0240: 100%|██████████| 1000/1000 [00:26<00:00, 37.15it/s]  


Best loss at iteration 972: 6.850/7.312/7.038


In [185]:
import models.BGNN
import sys
import importlib
importlib.reload(sys.modules['models.BGNN'])
from models.BGNN import BGNN



num_epochs = 200
patience = 50

k=20
l = 20
depth = 6
learning_rate = 0.01
gbdt_lr = 0.5

only_gbdt = False
train_non_gbdt = True


bgnn = BGNN(task='regression', depth=depth, heads=8, dropout=0., only_gbdt=only_gbdt, train_non_gbdt=train_non_gbdt, name='gat', lang='dgl', gbdt_lr=gbdt_lr)

min_rmse_epoch, metrics = bgnn.fit(networkx_graph, X, y, train_mask, val_mask, test_mask,
                                   num_epochs=num_epochs, patience=patience,
                                   hidden_dim=8, gbdt_trees_per_epoch=k, gnn_passes_per_epoch=l,
                                   cat_features=cat_features, learning_rate=learning_rate, logging_epochs=1, 
                                   loss_fn="", metric_name='loss', uncertainty=False)

# gnn.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='R2 RMSE', metric_name='accuracy', descending=False)
gnn.plot_interactive(metrics, legend=['Train', 'Val', 'Test'], title='BGNN RMSE', metric_name='loss')

Epoch 00053 | Loss 0.626 | Loss 0.631/7.954/7.731 | Time 0.8279:  26%|██▋       | 53/200 [00:45<02:06,  1.16it/s]

Best loss at iteration 2: 5.824/7.198/6.883
